In [1]:
import pandas as pd
import numpy as np
import os

In [3]:
df = pd.DataFrame()


for file_name in os.listdir('dotacie_pre'):
    if file_name.find('.csv') <= 0:
        continue
    year = int(file_name.replace('dotacie_ours_', '')[:4])
    if year in [2010, 2011, 2012]:
        delim, dec = ',', '.'
    else:
        delim, dec = ';', ','

    temp = pd.read_csv(f'dotacie_pre/{file_name}', delimiter=delim, decimal=dec)
    temp['rok'] = year
    df = pd.concat([df, temp], axis = 0)

for col in df.columns:
    if col == 'univerzita':
        continue
    df[col] = pd.to_numeric(df[col])

# display(df)
# display(df['univerzita'].value_counts())
# display(df['rok'].value_counts())

# df.to_csv('dotacie_pre/dotacie_ours_raw.csv', index = False)


In [33]:
facts = list(df.columns)
facts.remove('univerzita')
facts.remove('rok')
totals = (
    pd.concat(
        [df
        .groupby('rok')
        ['univerzita'].nunique()
        .to_frame(name = 'univerzity')]
        + [df
        .groupby('rok')
        [col].sum()
        .to_frame(name = col) for col in facts],
        axis = 1)
        .reset_index()
        )

totals

,rok,univerzity,dotacie,mzdy,dotacie-dokt,dotacie-stud,zamestnanci,int-dokt,dokt-abs,studenti,studenti-denni,publikacie,absolventi,ubyt-kap-sd,ubyt-studenti-sd,prispevok-sd,pub-podiel-07711,pub-podiel-07712
0,2009,20,3.829316e+08,172547633,23356608.0,50274862,22145.0,4321,1130.0,194554.0,137739.0,39033.0,50842.000000,46320.0,43901.0,3352848.0,0.0,0.0
1,2010,19,3.941945e+08,177193101,31593984.0,45552844,22570.0,5539,1535.0,187120.0,138507.0,85112.0,127768.632276,48175.0,44747.0,3326000.0,0.0,0.0
2,2011,19,3.925506e+08,122195435,33500216.0,47744810,0.0,6166,0.0,178779.0,134334.0,142607.0,85081.000000,0.0,0.0,0.0,0.0,0.0
3,2012,19,3.091253e+08,126633305,31613500.0,52043774,684.0,6165,0.0,178779.0,128222.0,205078.0,128737.885832,46140.0,44401.0,3288800.0,0.0,0.0
4,2013,20,4.103603e+08,126633305,23374620.0,53252028,0.0,6306,0.0,172823.0,133175.0,0.0,0.000000,45910.0,44094.0,3425888.0,100.0,100.0
5,2014,20,4.135335e+08,133523287,12127408.0,49175301,0.0,5930,0.0,165905.0,131347.0,0.0,0.000000,45961.0,43335.0,3358752.0,100.0,100.0
6,2015,20,4.215895e+08,135020294,3619605.0,50776959,0.0,5021,0.0,158110.5,117520.5,0.0,0.000000,45856.0,42390.0,3292528.0,100.0,100.0
7,2019,20,4.680454e+08,160556479,0.0,43218298,0.0,3568,0.0,122306.0,92452.0,0.0,0.000000,45088.0,39787.0,2901360.0,100.0,100.0
8,2020,20,4.680454e+08,160556479,0.0,43218298,0.0,3568,0.0,122306.0,92452.0,0.0,0.000000,45088.0,39787.0,2901360.0,100.0,100.0
9,2021,20,5.309199e+08,201937635,0.0,37361123,0.0,3454,0.0,115756.5,87814.5,0.0,32700.000000,44569.0,39278.0,2918064.0,100.0,100.0
